# 분류 (classification)

+ 머신러닝에서 회귀(regression)과 함께 가장 기본적인 데이타 분석 방법
+ 분류는 데이타가 어느 범주(category)에 해당하는지 판단하는 것이다.


## [1]  다항 분류

In [ ]:
# (1) 와인 데이터셋 불러오기
import pandas as pd
import pandas as pd
wine = pd.read_csv('./dataset/wine.csv', header=None)
wine.columns = ['fixed acidity','volatile acidity','citric acid','residual sugar','chlorides','free sulfur dioxide',
               'total sulfur dioxide','density','pH','sulphates','alcohol','quality',
               'type']
print(wine.shape)
wine.tail()

In [ ]:
# (2-1) 품질 데이터 확인
print(wine['quality'].describe())
print(wine['quality'].value_counts())

In [ ]:
# [확인] 품질 히스토그램 시각화
import matplotlib.pyplot as plt
plt.hist(wine['quality'], bins=7, rwidth=0.8)
plt.show()

In [ ]:
# (2-2) 품질을 3개의 범주(좋음, 보통, 나쁨)로 재분류
wine.loc[wine['quality'] <= 5, 'new_quality'] = 0
wine.loc[wine['quality'] == 6, 'new_quality'] = 1
wine.loc[wine['quality'] >= 7, 'new_quality'] = 2

print(wine['new_quality'].describe())
print(wine['new_quality'].value_counts())
# value_counts() : 항목수를 확인하며 어느 항목의 수가 너무 많거나 작으면 분류가 안 될 수 있기에 확인

In [ ]:
# (3) 데이터 정규화 및 train, test 데이터 분리

import tensorflow as tf

# new_quality 항목으로 대치하고자 quality 항목은 삭제한다.
del wine['quality']


wine_norm = (wine - wine.min()) / (wine.max() - wine.min())
wine_shuffle = wine_norm.sample(frac=1)
wine_np = wine_shuffle.to_numpy()

train_idx = int(len(wine_np) * 0.8)
train_X, train_Y = wine_np[:train_idx, :-1], wine_np[:train_idx, -1]
test_X, test_Y = wine_np[train_idx:, :-1], wine_np[train_idx:, -1]
train_Y = tf.keras.utils.to_categorical(train_Y, num_classes=3)
test_Y = tf.keras.utils.to_categorical(test_Y, num_classes=3)

In [ ]:
# (4) 와인 데이터셋 다항 분류 모델 생성 및 학습
#     마지막 레이어의 뉴런수가 3이다 ( new_quality=[0,1,2]) 



# [결과]
# Epoch 25/25
# 3897/3897 [==============================] - 0s 72us/sample - loss: 0.4110 - accuracy: 0.8052 - val_loss: 0.4051 - val_accuracy: 0.8023
# 약 80% 정도의 정확도이다.

In [ ]:
# [확인] 다항 분류 모델 학습 결과 시각화
import matplotlib.pyplot as plt
plt.figure(figsize=(12, 4))





In [ ]:
# (5) 다항 분류 모델 평가







# 이번에 테스트 데이트로 평가한 정확도와 위에서 검증 데이타의 정확도 80% 와 비교한다.
# 물론 여기서 더 좋은 성과를 내기 위해 다방면으로 노력해야 한다